<a href="https://colab.research.google.com/github/vfrantc/weather_experiments/blob/main/run_epdn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi 

Mon Jul 18 13:08:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!cp /content/drive/MyDrive/hazy_detect/EPDN.zip .

In [4]:
!unzip EPDN.zip

Archive:  EPDN.zip
   creating: EPDN/
  inflating: EPDN/run_epdn.ipynb     
  inflating: EPDN/.gitignore         
  inflating: EPDN/Enhanced_Pix2Pix_Dehazing_Network.pdf  
  inflating: EPDN/precompute_feature_maps.py  
  inflating: EPDN/readme.md          
  inflating: EPDN/test.py            
  inflating: EPDN/train.py           
   creating: EPDN/.git/
  inflating: EPDN/.git/index         
   creating: EPDN/.git/info/
  inflating: EPDN/.git/info/exclude  
  inflating: EPDN/.git/description   
   creating: EPDN/.git/branches/
   creating: EPDN/.git/hooks/
  inflating: EPDN/.git/hooks/pre-push.sample  
  inflating: EPDN/.git/hooks/fsmonitor-watchman.sample  
  inflating: EPDN/.git/hooks/commit-msg.sample  
  inflating: EPDN/.git/hooks/pre-merge-commit.sample  
  inflating: EPDN/.git/hooks/applypatch-msg.sample  
  inflating: EPDN/.git/hooks/pre-commit.sample  
  inflating: EPDN/.git/hooks/post-update.sample  
  inflating: EPDN/.git/hooks/prepare-commit-msg.sample  
  inflating: EPDN/.g

In [5]:
%cd EPDN

/content/EPDN


In [6]:
!pip install dominate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!cp /content/drive/MyDrive/weather_experiments/weather_test.zip .
!unzip weather_test.zip

Streaming output truncated to the last 5000 lines.
  inflating: test/outdoor/input/0194_1_0.2.jpg  
  inflating: test/outdoor/input/1986_0.9_0.08.jpg  
  inflating: test/outdoor/input/0251_0.8_0.16.jpg  
  inflating: test/outdoor/input/1953_0.8_0.2.jpg  
  inflating: test/outdoor/input/0127_0.8_0.2.jpg  
  inflating: test/outdoor/input/0003_0.8_0.2.jpg  
  inflating: test/outdoor/input/1861_0.85_0.2.jpg  
  inflating: test/outdoor/input/0317_0.85_0.08.jpg  
  inflating: test/outdoor/input/0269_0.95_0.2.jpg  
  inflating: test/outdoor/input/1055_0.95_0.2.jpg  
  inflating: test/outdoor/input/0267_0.9_0.16.jpg  
  inflating: test/outdoor/input/1848_0.8_0.12.jpg  
  inflating: test/outdoor/input/0082_0.85_0.16.jpg  
  inflating: test/outdoor/input/0138_0.9_0.08.jpg  
  inflating: test/outdoor/input/1724_0.95_0.2.jpg  
  inflating: test/outdoor/input/1889_0.85_0.08.jpg  
  inflating: test/outdoor/input/0302_0.9_0.16.jpg  
  inflating: test/outdoor/input/1868_1_0.08.jpg  
  inflating: test/

In [9]:
import os
import cv2
from glob import glob
from tqdm.notebook import tqdm 

In [12]:
def resize_images(folder):
    print('***[{}]***'.format(folder))
    for fname in tqdm(list(glob(os.path.join(folder, '*.png'))) + list(glob(os.path.join(folder, '*.jpg')))):
        image = cv2.imread(fname)
        h, w = image.shape[:2]

        if h > w:
            image = cv2.resize(image, (int(w * 640 / h), 640))
        else:
            image = cv2.resize(image, (640, int(h * 640 / w)))

        cv2.imwrite(fname, image)

In [11]:
!mv test/fog/input test/fog/mytest_A
!mv test/outdoor/input test/outdoor/mytest_A 
!mv test/real/input test/real/mytest_A 
!mv test/sand/input test/sand/mytest_A 
!mv test/test1/input test/test1/mytest_A

In [13]:
resize_images('test/fog/mytest_A')
resize_images('test/outdoor/mytest_A')
resize_images('test/real/mytest_A')
resize_images('test/sand/mytest_A')
resize_images('test/test1/mytest_A')

***[test/fog/mytest_A]***


  0%|          | 0/300 [00:00<?, ?it/s]

***[test/outdoor/mytest_A]***


  0%|          | 0/500 [00:00<?, ?it/s]

***[test/real/mytest_A]***


  0%|          | 0/497 [00:00<?, ?it/s]

***[test/sand/mytest_A]***


  0%|          | 0/323 [00:00<?, ?it/s]

***[test/test1/mytest_A]***


  0%|          | 0/750 [00:00<?, ?it/s]

In [27]:
!rm -rf /content/epdn
!mkdir /content/epdn

In [28]:
dataset = 'fog'
!rm -rf Pix2PixHD_modify/results/p2p/mytest_16/images
in_dir = '/content/EPDN/test/{}'.format(dataset)
model_path = '/content/EPDN/cpt'
!python3 /content/EPDN/test.py --which_epoch=16 --checkpoints_dir={model_path} --dataroot={in_dir}
!cp -r Pix2PixHD_modify/results/p2p/mytest_16/images /content/epdn/{dataset}

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: /content/EPDN/cpt
cluster_path: features_clustered_010.npy
dataroot: /content/EPDN/test/fog
display_winsize: 512
feat_num: 3
fineSize: 512
gpu_ids: [0]
how_many: 4469
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 512
load_features: False
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 3
n_blocks_local: 3
n_clusters: 10
n_colors: 3
n_downsample_E: 4
n_downsample_global: 4
n_feats: 32
n_local_enhancers: 1
n_resblocks: 8
name: p2p
nef: 16
netG: local
ngf: 16
niter_fix_global: 0
no_flip: False
no_instance: true
norm: instance
ntest: inf
output_nc: 3
phase: mytest
resize_or_crop: none
results_dir: Pix2PixHD_modify/results/
rgb_range: 255
scale: [1]
serial_batches: False
tf_log: False
use_dropout: False
which_epoch: 16
-------------- End ----------------
CustomDatasetDataLoader
dataset [AlignedDataset] was created
------------ Options -------------
batchSize: 1

In [29]:
dataset = 'outdoor'
!rm -rf Pix2PixHD_modify/results/p2p/mytest_16/images
in_dir = '/content/EPDN/test/{}'.format(dataset)
model_path = '/content/EPDN/cpt'
!python3 /content/EPDN/test.py --which_epoch=16 --checkpoints_dir={model_path} --dataroot={in_dir}
!cp -r Pix2PixHD_modify/results/p2p/mytest_16/images /content/epdn/{dataset}

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: /content/EPDN/cpt
cluster_path: features_clustered_010.npy
dataroot: /content/EPDN/test/outdoor
display_winsize: 512
feat_num: 3
fineSize: 512
gpu_ids: [0]
how_many: 4469
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 512
load_features: False
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 3
n_blocks_local: 3
n_clusters: 10
n_colors: 3
n_downsample_E: 4
n_downsample_global: 4
n_feats: 32
n_local_enhancers: 1
n_resblocks: 8
name: p2p
nef: 16
netG: local
ngf: 16
niter_fix_global: 0
no_flip: False
no_instance: true
norm: instance
ntest: inf
output_nc: 3
phase: mytest
resize_or_crop: none
results_dir: Pix2PixHD_modify/results/
rgb_range: 255
scale: [1]
serial_batches: False
tf_log: False
use_dropout: False
which_epoch: 16
-------------- End ----------------
CustomDatasetDataLoader
dataset [AlignedDataset] was created
------------ Options -------------
batchSiz

In [30]:
dataset = 'real'
!rm -rf Pix2PixHD_modify/results/p2p/mytest_16/images
in_dir = '/content/EPDN/test/{}'.format(dataset)
model_path = '/content/EPDN/cpt'
!python3 /content/EPDN/test.py --which_epoch=16 --checkpoints_dir={model_path} --dataroot={in_dir}
!cp -r Pix2PixHD_modify/results/p2p/mytest_16/images /content/epdn/{dataset}

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: /content/EPDN/cpt
cluster_path: features_clustered_010.npy
dataroot: /content/EPDN/test/real
display_winsize: 512
feat_num: 3
fineSize: 512
gpu_ids: [0]
how_many: 4469
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 512
load_features: False
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 3
n_blocks_local: 3
n_clusters: 10
n_colors: 3
n_downsample_E: 4
n_downsample_global: 4
n_feats: 32
n_local_enhancers: 1
n_resblocks: 8
name: p2p
nef: 16
netG: local
ngf: 16
niter_fix_global: 0
no_flip: False
no_instance: true
norm: instance
ntest: inf
output_nc: 3
phase: mytest
resize_or_crop: none
results_dir: Pix2PixHD_modify/results/
rgb_range: 255
scale: [1]
serial_batches: False
tf_log: False
use_dropout: False
which_epoch: 16
-------------- End ----------------
CustomDatasetDataLoader
dataset [AlignedDataset] was created
------------ Options -------------
batchSize: 

In [31]:
dataset = 'sand'
!rm -rf Pix2PixHD_modify/results/p2p/mytest_16/images
in_dir = '/content/EPDN/test/{}'.format(dataset)
model_path = '/content/EPDN/cpt'
!python3 /content/EPDN/test.py --which_epoch=16 --checkpoints_dir={model_path} --dataroot={in_dir}
!cp -r Pix2PixHD_modify/results/p2p/mytest_16/images /content/epdn/{dataset}

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: /content/EPDN/cpt
cluster_path: features_clustered_010.npy
dataroot: /content/EPDN/test/sand
display_winsize: 512
feat_num: 3
fineSize: 512
gpu_ids: [0]
how_many: 4469
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 512
load_features: False
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 3
n_blocks_local: 3
n_clusters: 10
n_colors: 3
n_downsample_E: 4
n_downsample_global: 4
n_feats: 32
n_local_enhancers: 1
n_resblocks: 8
name: p2p
nef: 16
netG: local
ngf: 16
niter_fix_global: 0
no_flip: False
no_instance: true
norm: instance
ntest: inf
output_nc: 3
phase: mytest
resize_or_crop: none
results_dir: Pix2PixHD_modify/results/
rgb_range: 255
scale: [1]
serial_batches: False
tf_log: False
use_dropout: False
which_epoch: 16
-------------- End ----------------
CustomDatasetDataLoader
dataset [AlignedDataset] was created
------------ Options -------------
batchSize: 

In [32]:
dataset = 'test1'
!rm -rf Pix2PixHD_modify/results/p2p/mytest_16/images
in_dir = '/content/EPDN/test/{}'.format(dataset)
model_path = '/content/EPDN/cpt'
!python3 /content/EPDN/test.py --which_epoch=16 --checkpoints_dir={model_path} --dataroot={in_dir}
!cp -r Pix2PixHD_modify/results/p2p/mytest_16/images /content/epdn/{dataset}

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: /content/EPDN/cpt
cluster_path: features_clustered_010.npy
dataroot: /content/EPDN/test/test1
display_winsize: 512
feat_num: 3
fineSize: 512
gpu_ids: [0]
how_many: 4469
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 512
load_features: False
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 3
n_blocks_local: 3
n_clusters: 10
n_colors: 3
n_downsample_E: 4
n_downsample_global: 4
n_feats: 32
n_local_enhancers: 1
n_resblocks: 8
name: p2p
nef: 16
netG: local
ngf: 16
niter_fix_global: 0
no_flip: False
no_instance: true
norm: instance
ntest: inf
output_nc: 3
phase: mytest
resize_or_crop: none
results_dir: Pix2PixHD_modify/results/
rgb_range: 255
scale: [1]
serial_batches: False
tf_log: False
use_dropout: False
which_epoch: 16
-------------- End ----------------
CustomDatasetDataLoader
dataset [AlignedDataset] was created
------------ Options -------------
batchSize:

In [33]:
import shutil
from shutil import copyfile
from shutil import move

def renam(folder):
  print('[{}]'.format(folder))
  for imname in tqdm(list(glob(os.path.join(folder, '*.png'))) + list(glob(os.path.join(folder, '*.jpg')))):
    new_name = imname.replace('_final.', '.')
    shutil.move(imname, new_name)

renam('/content/epdn/fog')
renam('/content/epdn/outdoor')
renam('/content/epdn/real')
renam('/content/epdn/sand')
renam('/content/epdn/test1')

[/content/epdn/fog]


  0%|          | 0/300 [00:00<?, ?it/s]

[/content/epdn/outdoor]


  0%|          | 0/500 [00:00<?, ?it/s]

[/content/epdn/real]


  0%|          | 0/497 [00:00<?, ?it/s]

[/content/epdn/sand]


  0%|          | 0/323 [00:00<?, ?it/s]

[/content/epdn/test1]


  0%|          | 0/750 [00:00<?, ?it/s]

In [34]:
!zip -r epdn.zip /content/epdn/
!cp epdn.zip /content/drive/MyDrive/weather_experiments

  adding: content/epdn/ (stored 0%)
  adding: content/epdn/outdoor/ (stored 0%)
  adding: content/epdn/outdoor/0081_0.8_0.2.png (deflated 0%)
  adding: content/epdn/outdoor/0240_1_0.08.png (deflated 0%)
  adding: content/epdn/outdoor/0024_0.85_0.08.png (deflated 0%)
  adding: content/epdn/outdoor/1869_1_0.2.png (deflated 0%)
  adding: content/epdn/outdoor/0355_0.85_0.12.png (deflated 0%)
  adding: content/epdn/outdoor/0308_1_0.16.png (deflated 0%)
  adding: content/epdn/outdoor/1015_0.8_0.12.png (deflated 0%)
  adding: content/epdn/outdoor/1853_0.85_0.08.png (deflated 0%)
  adding: content/epdn/outdoor/0171_0.85_0.12.png (deflated 0%)
  adding: content/epdn/outdoor/0361_0.85_0.2.png (deflated 0%)
  adding: content/epdn/outdoor/0372_0.85_0.12.png (deflated 0%)
  adding: content/epdn/outdoor/1038_1_0.16.png (deflated 0%)
  adding: content/epdn/outdoor/0055_0.85_0.12.png (deflated 0%)
  adding: content/epdn/outdoor/0092_0.85_0.2.png (deflated 0%)
  adding: content/epdn/outdoor/0351_0.95_0